In [ ]:

SELECT *,
CASE WHEN Frequency>1 THEN 1
    ELSE 0
  END AS Repeater,
 FROM (
SELECT z.customer_id,SUM(z.Recency) AS Recency,SUM(z.Frequency) AS Frequency,SUM(z.Monetary) AS Monetary,
SUM(z.Duration) AS Duration,SUM(z.Units) AS Units
FROM
(
SELECT y.customer_id, y.FY,#y.Division,
CASE WHEN y.FY="FY20" THEN DATE_DIFF(CAST("2020-04-01"AS DATE),MAX(DATE(y.Auth_Date)),DAY) 
     WHEN y.FY="FY21" THEN DATE_DIFF(CAST("2021-04-01"AS DATE),MAX(DATE(y.Auth_Date)),DAY)
     ELSE DATE_DIFF(CAST("2022-06-18"AS DATE),MAX(DATE(y.Auth_Date)),DAY)
END AS Recency, SUM(y.Frequency) AS Frequency, SUM(y.Monetary) AS Monetary,
CASE 
     WHEN (DATE_DIFF(MAX(DATE(y.Auth_Date)),MIN(DATE(y.Auth_Date)),DAY))=0 
     THEN   (IF(y.FY="FY20",DATE_DIFF(CAST("2020-04-01"AS DATE),MAX(DATE(y.Auth_Date)),DAY), 
             IF(y.FY="FY21", DATE_DIFF(CAST("2021-04-01"AS DATE),MAX(DATE(y.Auth_Date)),DAY)
            ,DATE_DIFF(CAST("2022-06-18"AS DATE),MAX(DATE(y.Auth_Date)),DAY))))
     WHEN y.FY="FY20" THEN DATE_DIFF(MAX(DATE(y.Auth_Date)),MIN(DATE(y.Auth_Date)),DAY)
     WHEN y.FY="FY21" THEN DATE_DIFF(MAX(DATE(y.Auth_Date)),MIN(DATE(y.Auth_Date)),DAY)
ELSE DATE_DIFF(MAX(DATE(y.Auth_Date)),MIN(DATE(y.Auth_Date)),DAY)
END AS Duration,
SUM(y.Units) AS Units
FROM 
(
SELECT
     customer_id,
     x.Auth_Date,
     CASE WHEN DATE(x.Auth_Date) >= '2019-04-01' AND DATE(x.Auth_Date) < '2020-04-01' THEN "FY20"
          WHEN DATE(x.Auth_Date) >= '2020-04-01' AND DATE(x.Auth_Date) < '2021-04-01' THEN "FY21"
          WHEN DATE(x.Auth_Date) >= '2021-04-01' AND DATE(x.Auth_Date) < '2022-04-01' THEN "FY22"
          WHEN DATE(x.Auth_Date) >= '2022-04-01' AND DATE(x.Auth_Date) < '2023-04-01' THEN "FY23"
      END AS FY,
      #FORMAT_DATE("%Y-%m",x.Auth_Date) AS YearMonth,
      #x.Division,
      /*x.Department,
       x.Level1,*/
      DATE_DIFF(CAST("2022-06-18" AS DATE),MAX(DATE(x.Auth_Date)),DAY) AS Recency,
      COUNT(DISTINCT x.order_id) AS Frequency,
      ROUND(SUM(x.order_item_sales),2) AS Monetary,
      IF((DATE_DIFF(MAX(DATE(x.Auth_Date)),MIN(DATE(x.Auth_Date)),DAY))=0,
      DATE_DIFF(CAST("2022-06-18" AS DATE),MAX(DATE(x.Auth_Date)),DAY),(DATE_DIFF(MAX(DATE(x.Auth_Date)),MIN(DATE(x.Auth_Date)),DAY))) AS Duration,
      SUM(x.Units) AS Units,

FROM (      

SELECT DISTINCT o.customer_id,
        #bh.bh_division AS Division, 
        /*bh.bh_reporting_department AS Department,
        bh.bh_level_1 AS Level1,*/
        DATE(o.order_authed_timestamp) AS Auth_Date,
        o.order_id,
        oi.order_item_sales,
        oi.order_item_quantity AS Units
    FROM `tal-production-data-bi.tal_dm_order.orders_2020_2030` o,
        UNNEST(o.order_items) oi
    INNER JOIN `tal-production-data-bi.tal_dm_product.business_hierarchy` bh
        ON bh.product_id = oi.product_id
    WHERE   
        DATE(o.order_placed_timestamp) >= "2022-01-01" AND DATE(o.order_placed_timestamp) < "2022-06-18" 
        AND DATE(o.order_authed_timestamp) >= "2022-01-01" AND DATE(o.order_authed_timestamp) < "2022-06-18" 
        AND o.order_authorisation_status IN ('Auth')
        AND o.is_not_return_cancel_in_full IS TRUE
        AND is_test_customer = false
        AND oi.order_item_status LIKE 'Shipped') x
    #where customer_id=2005339
    #AND Division = "Home"
    GROUP BY 1,2,3

    ORDER BY Auth_Date ASC) y
    WHERE y.FY IS NOT NULL #AND customer_id=5858791
    GROUP BY 1,2) z
    GROUP BY 1) p
  